!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

### Load the required Packages

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader,PyPDFLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub


from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os

c:\Users\anand\Rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
### Load the PDF file

In [3]:

data_file = PyPDFLoader("CG-1.pdf")
docs = data_file.load()

In [4]:
print(docs[0].page_content)

https://collegenote.pythonanywhere.com                                     Prepared By: Jayanta Poudel 
 
1 Computer Graphics (Reference Note)                                                                      BSc.CSIT                                                                                                   
Unit 1 
Introduction of Computer Graphics 
Computer graphics is a field related to the generation of graphics using computer. It includes 
the creation, storage and manipulation of images of object. These objects come from diverse 
field such as medicine, physical, mathematical, engineering, architecture, entertainment, 
advertisement. 
- It is related to the generation and the representation of graphics by a computer using 
specialized graphic hardware and software. The graphics can be photographs, drawings, 
movies, or simulation etc. 
- Computer graphics today is largely interactive; that is the user controls the contents 
structure and appearance of images of the obje

In [9]:
total_chars = sum(len(docs.page_content) for docs in docs)
print(total_chars)


31411


### Split Documents and Chunking

In [5]:
# create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                          chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [ ]:
len(chunks[1].page_content)


54

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "BAAI/bge-base-en-v1.5"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

C:\Users\anand\AppData\Local\Temp\ipykernel_482176\614034910.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


### VectorStore

In [11]:
# Vector store with the selected embedding model
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(chunks, embedding)

In [12]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [13]:
vectorstore_retreiver

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002C433E673D0>, search_kwargs={'k': 3})

In [15]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

BM25 Retriever is a sparse retrieval method that uses the BM25 (Best Matching 25) algorithm, which is a classic and highly effective information retrieval technique.

What is BM25?
BM25 is a bag-of-words retrieval function that ranks documents based on query terms appearing in each document. It's considered one of the most effective traditional retrieval algorithms.

How BM25 Works
Unlike vector stores that use dense embeddings, BM25:

Sparse Representation: Uses term frequency and document statistics

Keyword-based: Relies on exact word matching (but with smart weighting)

Statistical Approach: Considers:

Term frequency (TF) in documents

Inverse document frequency (IDF)

Document length normalization

### Ensemble Retriever

In [16]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
    temperature=0.7)

### Prompt Template:

In [19]:
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [20]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [21]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [27]:
print(chain.invoke(" If a pixel is accessed from the frame buffer with an average access time of 300ns then will this rate produce an un-flicking effect for the screen size of 640 × 480. "))

To determine if the given rate will produce an un-flicking effect, we need to calculate the total time required to access all pixels on the screen and compare it to the typical refresh rate needed to avoid flickering.

1.  **Calculate the total number of pixels on the screen:**
    Screen size = 640 × 480
    Total no. of pixels = 640 * 480 = 307,200 pixels

2.  **Calculate the total time required to access all pixels for one frame:**
    Average access time of one pixel = 300 ns
    Total time = Total no. of pixels * Average access time per pixel
    Total time = 307,200 * 300 ns = 92,160,000 ns

3.  **Convert the total time to seconds:**
    1 second = 10^9 ns
    Total time = 92,160,000 / 10^9 seconds = 0.09216 seconds

4.  **Compare with the required refresh rate for an un-flicking effect:**
    To produce an un-flicking effect, a screen typically needs a refresh rate of at least 60 frames per second (Hz).
    This means the time taken to refresh one frame should be:
    Time per f

In [ ]:
print(chain.invoke("How does Orca compares to ChatGPT?"))

Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question. 

Think step by step before answering the question. You will get a $100 tip if you provide correct answer. 

CONTEXT: [Document(page_content='While significantly better than Vicuna and marginally better than ChatGPT, Orca’s average performance of 49.7%, lags GPT-4 by 26%. Note that GPT-4 has reported a data contami- nation issue with Big-Bench and that we are not aware of such issues with either LLaMA’s training data (the base model used by both Vicuna and Orca) or the Flan-V2 collection or Vicuna’s training data (ShareGPT).\n\nGiven the close performance on average on BigBench-Hard, we take a deeper look at differences in performance between Orca and ChatGPT:\n\nEntailment and Semantic Understanding:\n\nOrca performs better at entailment (formal fallacies) and semantic understanding (Dis- ambiguation QA and Snarks).', metadata={'source': './da

In [2]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

# Initialize the model
llm = ChatGroq(model="llama3-8b-8192", temperature=0.7)


ModuleNotFoundError: No module named 'langchain_groq'

Note: you may need to restart the kernel to use updated packages.


c:\Users\anand\Rag\.venv\Scripts\python.exe: No module named pip
